In [1]:
pip install tushare

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tushare as ts
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sb

import dataset

In [3]:
ts.set_token('293d683e83b36fad82472d9a4b4c3903d21fa6bcb85cb0cd4f4b9414')
pro = ts.pro_api()
df = pro.daily(ts_code='000001.SZ', start_date='2010-01-01', end_date='2022-12-31')

In [4]:
df = df.iloc[::-1]
df.reset_index(inplace=True)

In [5]:
training_set = df.loc[:, ['close']]
training_set = training_set.values
'''from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set = sc.fit_transform(training_set)'''

'from sklearn.preprocessing import MinMaxScaler\nsc = MinMaxScaler(feature_range = (0, 1))\ntraining_set = sc.fit_transform(training_set)'

form a list of records of every 60 intervals of past closing price

In [6]:
from sklearn.preprocessing import scale
x_train = []
y_train = []
for i in range(60, len(training_set)):
    x_train.append(training_set[i-60:i])
    y_train.append(training_set[i, training_set.shape[1] - 1])
x, y = np.array(x_train), np.array(y_train)
print(x.shape)
print(y.shape)

(2756, 60, 1)
(2756,)


In [7]:
device= torch.device("cpu")
print(device)

cpu


set first 2000 data as training set and the rest and testing set

In [8]:
x = torch.from_numpy(np.asarray(x).astype(np.float32))
y = torch.from_numpy(np.asarray(y).astype(np.float32))
x_train = x[:2000]
y_train = y[:2000]
x_test = x[2000:]
y_test = y[2000:]


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([2000, 60, 1])
torch.Size([2000])
torch.Size([756, 60, 1])
torch.Size([756])


In [9]:
def eval_on_test_set():

    temp_error=0
    num=0

    for i in range(0,750,batch_size):

        data =  x_test[i:i+batch_size]
        label= y_test[i:i+batch_size]

        data=data.to(device)
        label=label.to(device)
        
        inputs = data.view(batch_size,60)

        scores=net( inputs ) 

        error = get_error( scores , label)

        temp_error += error

        num+=1


    total_error = temp_error/num
    print( f'Total error on testing set : {total_error}')

In [10]:
def get_error( scores , labels ):

    bs=scores.size(0)
    matches = 0
    
    for i in range (len(scores)):
        if abs(scores[i] - labels[i]) <=1.5:
            matches+=1
    
    return 1-matches/batch_size

The MLP class

In [11]:
class MLPRegression(nn.Module):
    def __init__(self, dim=1024):
        super(MLPRegression,self).__init__()

        self.hidden1 = nn.Sequential(
                        
                        nn.Linear(x.shape[1], dim),
                        nn.BatchNorm1d(dim),
                        nn.Linear(dim, dim),
                        nn.BatchNorm1d(dim),
                        nn.ReLU(inplace=True),
                        nn.Dropout(0.2),
                        )
         
         
        self.hidden2 = nn.Sequential(
                        nn.Linear(dim, dim),
                        nn.BatchNorm1d(dim),
                        nn.Linear(dim, dim),
                        nn.BatchNorm1d(dim),
                        nn.ReLU(inplace=True),
                        nn.Dropout(0.2),
                        )
        self.hidden3 = nn.Sequential(
                        nn.Linear(dim, dim),
                        nn.BatchNorm1d(dim),
                        nn.Linear(dim, dim),
                        nn.BatchNorm1d(dim),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        )
        
   
        self.predict=nn.Linear(dim,1)
        
        
    def forward(self,x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output=self.predict(x)
        return output[:,0]
    
net = MLPRegression()
batch_size = 10
lr = 0.01
net = net.to(device)

In [ ]:
criterion = nn.MSELoss()

for epoch in range(1,100):
    
    if epoch%7 == 0:
        lr = lr / 1.5
        
    optimizer = torch.optim.SGD(net.parameters(), lr=lr) 
    temp_loss=0
    temp_error = 0
    num =0
        
    random =torch.randperm(2000)
    
    for count in range(0,2000,batch_size):
        optimizer.zero_grad() 
        indices=random[count:count+batch_size]
       
        data =  x_train[indices].to(device)
        

        label= y_train[indices].to(device)
        
   
        inputs=data.view(batch_size,60)
       

        inputs.requires_grad_()
        predict=net(inputs)


        loss = criterion(predict, label)  

        loss.backward()
        optimizer.step()
        
        temp_loss += loss.detach().item()
        temp_error += get_error( predict.detach() ,label)
        
        num +=1  
        
    if epoch == 99:
            torch.save(net.state_dict(), f'./close.pth')
    total_loss = temp_loss/num
    total_error = temp_error/num
    print(f"Epoch: {epoch} Learning rate: {lr} Total Loss: {total_loss} Total Error: {total_error}" )
    eval_on_test_set()

Epoch: 1 Learning rate: 0.01 Total Loss: 240.5203209877014 Total Error: 0.6119999999999998
Total error on testing set : 0.8053333333333328
Epoch: 2 Learning rate: 0.01 Total Loss: 5.922253388166427 Total Error: 0.4795000000000003
Total error on testing set : 0.7533333333333331
Epoch: 3 Learning rate: 0.01 Total Loss: 5.238962596058846 Total Error: 0.45900000000000013
Total error on testing set : 0.7773333333333331
Epoch: 4 Learning rate: 0.01 Total Loss: 4.638727896213531 Total Error: 0.42149999999999976
Total error on testing set : 0.757333333333333
Epoch: 5 Learning rate: 0.01 Total Loss: 3.956822547316551 Total Error: 0.3889999999999999
Total error on testing set : 0.8053333333333332
Epoch: 6 Learning rate: 0.01 Total Loss: 3.7818360093235968 Total Error: 0.3725
Total error on testing set : 0.7973333333333332
Epoch: 7 Learning rate: 0.006666666666666667 Total Loss: 3.3210417038202285 Total Error: 0.34600000000000025
Total error on testing set : 0.7719999999999998
Epoch: 8 Learning r

print the predicton results

In [ ]:
device= torch.device("cpu")
print(device)
net = MLPRegression()
net.load_state_dict(torch.load('./close.pth',map_location=torch.device('cpu')))
print(net.hidden1.parameters)
print(net.hidden1[0].bias.data)

net.hidden3[1].bias.data = net.hidden3[1].bias.data *1.8
net.hidden3[3].bias.data = net.hidden3[3].bias.data * 1.8

temp = x_test.squeeze()
temp = temp.view(x_test.shape[0],60)

predicted = net(temp.to(device))
plt.axes([0,0,3,2])
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.plot(y_test,linewidth=3,label='Label Price')
plt.plot(predicted.detach().numpy(),alpha=0.8,label='Predicted Price')
plt.legend()